# Todo
* change fitness

# Setup

In [ ]:
!nvidia-smi

Mon Jun 14 08:27:51 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%pip install -q wandb
import wandb

     |████████████████████████████████| 1.8MB 7.1MB/s 
     |████████████████████████████████| 133kB 47.3MB/s 
     |████████████████████████████████| 174kB 46.2MB/s 
     |████████████████████████████████| 102kB 14.6MB/s 
     |████████████████████████████████| 71kB 10.8MB/s 


In [ ]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
import xml.etree.ElementTree as ET
import os
import PIL
import shutil
import random
import numpy as np
from pathlib import Path
from PIL import Image, ImageDraw

In [ ]:
ALL_CLASSES = ["aquarium", "bottle", "bowl", "box", "bucket", "plastic_bag", "plate", "styrofoam", "tire", "toilet", "tub", "washing_machine", "water_tower"]
VAL_CLASSES = ALL_CLASSES
TRAIN_CLASSES = ALL_CLASSES
TRAIN_SIZE = 0.99
random.seed(601)

In [ ]:
# train dataset
!cp drive/MyDrive/作業/深度學習概論與應用/cdc/train_cdc.zip .
!unzip -q train_cdc.zip
# test dataset
!cp drive/MyDrive/作業/深度學習概論與應用/cdc/test_cdc.zip .
!unzip -q test_cdc.zip
# extra dataset
!cp drive/MyDrive/作業/深度學習概論與應用/cdc/extra.zip .
!unzip -q extra.zip
# sticker
!cp drive/MyDrive/作業/深度學習概論與應用/cdc/sticker.zip .
!unzip -q sticker.zip

In [ ]:
!git clone https://github.com/lashhw/yolov5
!pip install -q -r yolov5/requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 7027, done.
remote: Counting objects: 100% (133/133), done.
remote: Compressing objects: 100% (80/80), done.
remote: Total 7027 (delta 73), reused 101 (delta 53), pack-reused 6894
Receiving objects: 100% (7027/7027), 9.08 MiB | 33.46 MiB/s, done.
Resolving deltas: 100% (4809/4809), done.
     |████████████████████████████████| 645kB 37.7MB/s 


# Train

In [ ]:
def xxyy2xywh(img_w, img_h, box):
  dw = 1./img_w
  dh = 1./img_h
  x = (box[0] + box[1])/2.0
  y = (box[2] + box[3])/2.0
  w = box[1] - box[0]
  h = box[3] - box[2]
  x = x*dw
  w = w*dw
  y = y*dh
  h = h*dh
  return (x,y,w,h)

def voc2yolo(image_folder, annotation_folder, output_path, train_val_split=False):
  _, _, fns = next(os.walk(image_folder)) # get filenames

  Path('{}/images/train'.format(output_path)).mkdir(parents=True, exist_ok=True)
  Path('{}/images/val'.format(output_path)).mkdir(parents=True, exist_ok=True)
  Path('{}/labels/train'.format(output_path)).mkdir(parents=True, exist_ok=True)
  Path('{}/labels/val'.format(output_path)).mkdir(parents=True, exist_ok=True)
    
  for fn in fns:
    dataset_type = 'train'
    if train_val_split and random.random() > TRAIN_SIZE:
      dataset_type = 'val'
    shutil.copy2('{}/{}'.format(image_folder, fn), '{}/images/{}'.format(output_path, dataset_type))

    # parse data
    fn_wo_ext = os.path.splitext(fn)[0] # filenames without extension
    annotation_path = '{}/{}.xml'.format(annotation_folder, fn_wo_ext)
    yolo_annotation_str = ''
    if os.path.isfile(annotation_path):
      in_file = open(annotation_path)
      tree = ET.parse(in_file)
      root = tree.getroot()
      size = root.find('size')
      w = int(size.find('width').text)
      h = int(size.find('height').text)
      # iterate object
      for obj in root.iter('object'):
        cls = obj.find('name').text
        if cls not in TRAIN_CLASSES:
          print('{}/{}: {} ignored.'.format(image_folder, fn, cls))
          continue
        if cls not in VAL_CLASSES: # not suitable for validation set
          dataset_type = 'train'
        cls_id = TRAIN_CLASSES.index(cls)
        xmlbox = obj.find('bndbox')
        b = tuple(float(xmlbox.find(x).text) for x in ['xmin', 'xmax', 'ymin', 'ymax'])
        bb = xxyy2xywh(w, h, b)
        if (yolo_annotation_str != ''): yolo_annotation_str += '\n'
        yolo_annotation_str += str(cls_id) + ' '
        yolo_annotation_str += ' '.join([str(a) for a in bb])
    with open('{}/labels/{}/{}.txt'.format(output_path, dataset_type, fn_wo_ext), 'w') as out_file:
      out_file.write(yolo_annotation_str)

# convert extra dataset
root, dirs, _ = next(os.walk('extra/images'))
for dir in dirs:
  voc2yolo('{}/{}'.format(root, dir),
           'extra/annotations',
           'extra_cdc',
           True)
# convert train dataset
voc2yolo('train_cdc/train_images',
         'train_cdc/train_annotations',
         'train_cdc',
         True)

In [ ]:
preset = '''train: [/content/train_cdc/images/train, /content/extra_cdc/images/train]
val: [/content/train_cdc/images/val, /content/extra_cdc/images/val]
nc: {nc}
names: {names}'''
names = '["' + '", "'.join(TRAIN_CLASSES) + '"]'
with open('dataset.yaml', 'w') as file:
  file.write(preset.format(nc=len(TRAIN_CLASSES), names=names))

In [ ]:
%cd /content/yolov5
!python train.py --img 832 --batch 7 --epochs 55 \
                 --weights yolov5l6.pt \
                 --hyp /content/yolov5/data/hyp.finetune_objects365.yaml \
                 --data /content/dataset.yaml

/content/yolov5
github: up to date with https://github.com/lashhw/yolov5 ✅
YOLOv5 🚀 v5.0-170-gf5b908f torch 1.8.1+cu101 CUDA:0 (Tesla T4, 15109.75MB)

Namespace(adam=False, artifact_alias='latest', batch_size=7, bbox_interval=-1, bucket='', cache_images=False, cfg='', data='/content/dataset.yaml', device='', entity=None, epochs=55, evolve=False, exist_ok=False, global_rank=-1, hyp='/content/yolov5/data/hyp.finetune_objects365.yaml', image_weights=False, img_size=[832, 832], label_smoothing=0.0, linear_lr=False, local_rank=-1, multi_scale=False, name='exp', noautoanchor=False, nosave=False, notest=False, project='runs/train', quad=False, rect=False, resume=False, save_dir='runs/train/exp', save_period=-1, single_cls=False, sync_bn=False, total_batch_size=7, upload_dataset=False, weights='yolov5l6.pt', workers=8, world_size=1)
tensorboard: Start with 'tensorboard --logdir runs/train', view at http://localhost:6006/
2021-06-14 04:02:03.342420: I tensorflow/stream_executor/platform/default

In [ ]:
from datetime import datetime
%cd /content
time = datetime.today().strftime('%Y%m%d-%H%M')
shutil.copy2('yolov5/runs/train/exp/weights/last.pt', 'drive/MyDrive/作業/深度學習概論與應用/cdc/weights/%s_last.pt'%(time))
# shutil.copy2('yolov5/runs/train/exp/weights/best.pt', 'drive/MyDrive/作業/深度學習概論與應用/cdc/weights/%s_best.pt'%(time))

/content


'drive/MyDrive/作業/深度學習概論與應用/cdc/weights/20210614-0734_last.pt'

# Detect

In [ ]:
%cd /content/yolov5
!python test.py --conf-thres 0.001 --data /content/dataset.yaml --weights /content/yolov5/runs/train/exp/weights/best.pt

/content/yolov5
Namespace(augment=False, batch_size=32, conf_thres=0.001, data='/content/dataset.yaml', device='', exist_ok=False, half=False, img_size=640, iou_thres=0.6, name='exp', project='runs/test', save_conf=False, save_hybrid=False, save_json=False, save_txt=False, single_cls=False, task='val', verbose=False, weights=['/content/yolov5/runs/train/exp/weights/best.pt'])
YOLOv5 🚀 v5.0-170-gf5b908f torch 1.8.1+cu101 CUDA:0 (Tesla T4, 15109.75MB)

Fusing layers... 
Model Summary: 501 layers, 76703256 parameters, 0 gradients, 116.5 GFLOPs
val: Scanning '/content/extra_cdc/labels/val.cache' images and labels... 719 found, 0 missing, 21 empty, 0 corrupted: 100% 719/719 [00:00<00:00, 7712799.43it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 23/23 [00:28<00:00,  1.23s/it]
                 all        719       1987      0.715      0.547      0.593      0.462
            aquarium        719         29      0.831      0.724      0.818      

In [ ]:
%cd /content/yolov5
!python detect.py --img 640 --conf-thres 0.00001 \
                  --source /content/test_cdc/test_images \
                  --weights /content/yolov5/runs/train/exp/weights/best.pt \
                  --save-txt --save-conf --nosave

/content/yolov5
Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=1e-05, device='', exist_ok=False, half=False, hide_conf=False, hide_labels=False, img_size=640, iou_thres=0.45, line_thickness=3, max_det=1000, name='exp', nosave=True, project='runs/detect', save_conf=True, save_crop=False, save_txt=True, source='/content/test_cdc/test_images', update=False, view_img=False, weights=['/content/yolov5/runs/train/exp/weights/best.pt'])
YOLOv5 🚀 v5.0-170-gf5b908f torch 1.8.1+cu101 CUDA:0 (Tesla T4, 15109.75MB)

Fusing layers... 
Model Summary: 501 layers, 76703256 parameters, 0 gradients, 116.5 GFLOPs
image 1/2248 /content/test_cdc/test_images/200801213.jpg: 512x640 2 aquariums, 12 bottles, 38 bowls, 1 box, 74 buckets, 15 plastic_bags, 5 plates, 59 tires, 8 toilets, 11 water_towers, Done. (0.059s)
image 2/2248 /content/test_cdc/test_images/200801216.jpg: 512x640 3 aquariums, 22 bottles, 36 bowls, 28 boxs, 22 buckets, 27 plastic_bags, 5 plates, 33 styrofoams, 8 tires, 3 t

In [ ]:
from PIL import Image
%cd /content
label_dir = 'yolov5/runs/detect/exp'
with open('submission.csv', 'w') as csv_file:
  csv_file.write('image_filename,label_id,x,y,w,h,confidence\n')

  _, _, filenames = next(os.walk(label_dir+'/labels'))
  image_ids = [os.path.splitext(filename)[0] for filename in filenames]

  images_with_annotation = []
  for image_id in image_ids:
    with open(label_dir+'/labels/%s.txt'%(image_id)) as file:
      lines = file.readlines()
    annotations = []
    for line in lines:
      data = line.split()
      im = Image.open('test_cdc/test_images/%s.jpg'%(image_id))
      w, h = im.size
      x_center = w * float(data[1])
      y_center = h * float(data[2])
      width = max(0, w * float(data[3]))
      height = max(0, h * float(data[4]))
      x_left = max(0, x_center - width/2)
      y_top = max(0, y_center - height/2)
      cls = int(data[0])
      conf = data[5]
      # for drawing rectangle
      if float(conf) >= 0.25:
        annotations.append((ALL_CLASSES[cls], (x_left, y_top, x_left+width, y_top+height), conf))
      # write to submission.csv
      result = '{}.jpg,{},{:.1f},{:.1f},{:.1f},{:.1f},{}\n'.format(
                image_id, cls+1, x_left, y_top, width, height, conf)
      csv_file.write(result)
    if len(annotations) > 0:
      images_with_annotation.append((image_id, annotations))

/content


In [ ]:
for img_id, annotations in images_with_annotation:
  source_img = Image.open('test_cdc/test_images/{}.jpg'.format(img_id)).convert('RGB')
  img = {x: source_img.copy() for x in ALL_CLASSES}
  draw = {x: ImageDraw.Draw(img[x]) for x in ALL_CLASSES}
  exist = set()
  for cls, bbox, conf in annotations:
    draw[cls].rectangle(bbox, outline='red')
    draw[cls].text((bbox[0], bbox[1]), '{:.2f}'.format(float(conf)), fill='red')
    exist.add(cls)
  for cls in exist:
    dir_path = 'image_by_class/{}'.format(cls)
    Path(dir_path).mkdir(parents=True, exist_ok=True)
    img[cls].save('{}/{}.jpg'.format(dir_path, img_id))

In [ ]:
!cp -r /content/image_by_class /content/drive/MyDrive/作業/深度學習概論與應用/cdc/image_by_class_pred/0612-0.61

In [ ]:
%cd yolov5
!python detect.py --img 640 --line-thickness 1 \
                  --source /content/test_cdc/test_images \
                  --weights /content/yolov5/runs/train/exp/weights/best.pt

/content/yolov5
Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.25, device='', exist_ok=False, half=False, hide_conf=False, hide_labels=False, img_size=640, iou_thres=0.45, line_thickness=1, max_det=1000, name='exp', nosave=False, project='runs/detect', save_conf=False, save_crop=False, save_txt=False, source='/content/test_cdc/test_images', update=False, view_img=False, weights=['/content/yolov5/runs/train/exp/weights/best.pt'])
YOLOv5 🚀 v5.0-168-g97c4422 torch 1.8.1+cu101 CUDA:0 (Tesla T4, 15109.75MB)

Fusing layers... 
Model Summary: 501 layers, 76703256 parameters, 0 gradients, 116.5 GFLOPs
image 1/2248 /content/test_cdc/test_images/200801213.jpg: 512x640 1 bowl, 3 buckets, Done. (0.049s)
image 2/2248 /content/test_cdc/test_images/200801216.jpg: 512x640 Done. (0.041s)
image 3/2248 /content/test_cdc/test_images/200801221.jpg: 512x640 1 styrofoam, Done. (0.041s)
image 4/2248 /content/test_cdc/test_images/20080123.jpg: 512x640 1 bucket, Done. (0.042s)
image 5/2

In [ ]:
!cp -r /content/yolov5/runs/detect/exp2 /content/drive/MyDrive/作業/深度學習概論與應用/cdc/exp/0612-0.61

# Additional

In [ ]:
!cp -r /content/yolov5/runs/detect/exp /content/drive/MyDrive

In [ ]:
%cd /content/yolov5
!python train.py --img 438 --batch 40 --epochs 50 --data /content/train_cdc/dataset.yaml --weights /content/yolov5/runs/train/exp/weights/last.pt --cache

In [ ]:
%cd /content/yolov5
!python test.py  --img 438 --conf-thres 0.00001 --weights /content/yolov5/runs/train/exp/weights/best.pt --data /content/train_cdc/dataset.yaml

/content/yolov5
Namespace(augment=False, batch_size=32, conf_thres=1e-05, data='/content/train_cdc/dataset.yaml', device='', exist_ok=False, img_size=438, iou_thres=0.6, name='exp', project='runs/test', save_conf=False, save_hybrid=False, save_json=False, save_txt=False, single_cls=False, task='val', verbose=False, weights=['/content/yolov5/runs/train/exp/weights/best.pt'])
YOLOv5 🚀 v5.0-128-g4b52e19 torch 1.8.1+cu101 CUDA:0 (Tesla T4, 15109.75MB)

Fusing layers... 
Model Summary: 308 layers, 21065925 parameters, 0 gradients, 50.4 GFLOPS
val: Scanning '/content/train_cdc/labels/val.cache' images and labels... 244 found, 0 missing, 0 empty, 0 corrupted: 100% 244/244 [00:00<00:00, 2210389.15it/s]
               Class      Images      Labels           P           R      mAP@.5  mAP@.5:.95: 100% 8/8 [00:04<00:00,  1.96it/s]
                 all         244         686       0.614       0.491       0.531       0.393
              bottle         244          60       0.356      0.0833       